# Parte 1: Lectura de datos con diferentes formatos

In [1]:
%reset -f

In [6]:
#pip install et-xmlfile

In [1]:
import pandas as pd
import numpy as np
import os
from datetime import datetime
# Librería para tratamiento de ficheros xml
import xml.etree.ElementTree as ET
# Librería para efectuar peticiones a páginas web
import requests


Vamos a generar las funciones necesarias para cargar los datos en los diferentes formatos

https://pandas.pydata.org/docs/search.html?q=read_

https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html

https://pandas.pydata.org/docs/reference/api/pandas.read_json.html

https://pandas.pydata.org/docs/reference/api/pandas.read_xml.html

https://pandas.pydata.org/docs/reference/api/pandas.read_excel.html

In [2]:
# carga de archivos xml

def cargaXml():
    inicio = datetime.now()
    xml_data = open('./data/calendario_laboral_2022.xml', 'r', encoding="UTF-8").read()  # Leer archivo
    root = ET.XML(xml_data)  # Parsear el XML
    data = []
    cols = []
    for i, child in enumerate(root):
        data.append([subchild.text for subchild in child])
        cols.append(child.tag)

    df = pd.DataFrame(data).T  # Escribir en DF y transponer datos
    df.columns = cols  # Actualizamos nombres de columna Update column names
    
    return ((datetime.now()-inicio).total_seconds())

In [3]:
# carga de archivos xlsx
def cargaXlsx ():
    inicio = datetime.now()
    xlsx_data = pd.read_excel('./data/calendario_laboral_2022.xlsx', sheet_name='calendario_laboral_2022')
    return ((datetime.now()-inicio).total_seconds())

In [4]:
# carga de archivos csv
def cargaCsv ():
    inicio = datetime.now()
    csv_data = pd.read_csv('./data/calendario_laboral_2022.csv', sep=';', encoding = "latin-1")
    return ((datetime.now()-inicio).total_seconds())

In [5]:
# carga de archivos json
def cargaJson ():
    inicio = datetime.now()
    json_data = pd.read_json('./data/calendario_laboral_2022.json')
    return ((datetime.now()-inicio).total_seconds())

In [6]:
cargaXml()

0.016003

In [7]:
cargaXlsx()

1.729524

In [8]:
cargaCsv()

0.00726

In [9]:
cargaJson()

0.024913

Vemos que la carga de datos más rápida es a través de CSV's.  También podemos cargar archivos directamente de internet...

In [10]:
# carga de archivos csv internet
def cargaCsvInternet ():
    inicio = datetime.now()
    csv_data = pd.read_csv('https://opendata.euskadi.eus/contenidos/ds_eventos/calendario_laboral_2022/opendata/calendario_laboral_2022.csv', sep=';', encoding="latin-1")
    return ((datetime.now()-inicio).total_seconds())

In [11]:
cargaCsvInternet()

0.19169

Lo ponemos bonito y automatizamos... Ejecutaremos el proceso 5 veces para tener un tiempo medio de carga por tipo de archivo

In [27]:
resultados

,0,1,2,3,4,5
xml,NaN,NaN,NaN,NaN,NaN,0.009443
xlsx,NaN,NaN,NaN,NaN,NaN,0.111258
csv,NaN,NaN,NaN,NaN,NaN,0.009038
json,NaN,NaN,NaN,NaN,NaN,0.010973
csvInternet,NaN,NaN,NaN,NaN,NaN,0.868178


In [33]:
tiempos = pd.DataFrame([xml, xlsx, csv, json, csvInternet])
tiempos

,0
0,0.008002
1,0.177705
2,0.007113
3,0.013684
4,0.840423


In [29]:
    print ('Iteración:', a)
    xml = cargaXml()
    xlsx = cargaXlsx()
    csv = cargaCsv()
    json = cargaJson()
    csvInternet = cargaCsvInternet()

Iteración: 0


In [43]:
resultados = pd.DataFrame(columns=['xml','xlsx','csv','json','csvInternet'])
tiempos = np.array([])
for a in range(5):
    print ('Iteración:', a)
    xml = cargaXml()
    xlsx = cargaXlsx()
    csv = cargaCsv()
    json = cargaJson()
    csvInternet = cargaCsvInternet()
    
    tiempos = pd.DataFrame([xml, xlsx, csv, json, csvInternet]).transpose()
    tiempos.columns = ['xml','xlsx','csv','json','csvInternet']
   
    resultados = pd.concat([resultados, tiempos], ignore_index=True, axis=0)


Iteración: 0
Iteración: 1
Iteración: 2
Iteración: 3
Iteración: 4


In [44]:
resultados

,xml,xlsx,csv,json,csvInternet
0,0.106627,0.103932,0.011214,0.016393,0.243431
1,0.006906,0.132369,0.006998,0.006903,0.418762
2,0.008015,0.135393,0.000000,0.016003,0.405415
3,0.008130,0.097475,0.007016,0.013927,0.131502
4,0.007185,0.104256,0.006919,0.006749,0.111988


Calculamos los tiempos medios para cada tipo de archivo

In [45]:
tiemposMedios = pd.DataFrame(columns=['xml','xlsx','csv','json','csvInternet'])

tiemposMedios.loc ['0','xml'] = round(resultados ['xml'].mean(),3)
tiemposMedios.loc ['0','xlsx'] = round(resultados ['xlsx'].mean(),3)
tiemposMedios.loc ['0','csv'] = round(resultados ['csv'].mean(),3)
tiemposMedios.loc ['0','json'] = round(resultados ['json'].mean(),3)
tiemposMedios.loc ['0','csvInternet'] = round(resultados ['csvInternet'].mean(),3)

In [46]:
tiemposMedios

,xml,xlsx,csv,json,csvInternet
0,0.027,0.115,0.006,0.012,0.262


También podemos cargar tablas desde una url en internet...

In [47]:
url = 'http://www.ffiec.gov/census/report.aspx?year=2011&state=01&report=demographic&msa=11500'
html = requests.get(url).content
df_list = pd.read_html(html)
df = df_list[-1] # Nos quedamos con la ultima tabla descargada

df.to_csv('tabla descargada de internet.csv')
df

,County Code,Tract Code,Tract Income Level,Distressed or Under -served Tract,Tract Median Family Income %,2011 HUD Est. MSA/MD non-MSA/MD Median Family Income,2011 Est. Tract Median Family Income,2000 Tract Median Family Income,Tract Population,Tract Minority %,Minority Population,Owner Occupied Units,1- to 4- Family Units
0,15,1.00,Moderate,No,74.23,"$51,500","$38,228","$29,615",2279,45.77,1043,431,894
1,15,2.00,Middle,No,86.40,"$51,500","$44,496","$34,468",3084,42.44,1309,881,1543
2,15,3.00,Moderate,No,59.27,"$51,500","$30,524","$23,644",3545,78.87,2796,637,1562
3,15,4.00,Moderate,No,76.24,"$51,500","$39,264","$30,417",2777,59.42,1650,908,1335
4,15,5.00,Low,No,40.54,"$51,500","$20,878","$16,172",2372,93.55,2219,597,1306
5,15,6.00,Low,No,30.36,"$51,500","$15,635","$12,113",2439,77.74,1896,363,736
6,15,8.00,Moderate,No,72.62,"$51,500","$37,399","$28,971",1475,55.05,812,293,674
7,15,9.00,Upper,No,147.52,"$51,500","$75,973","$58,854",3705,16.22,601,1214,1606
8,15,10.00,Upper,No,142.34,"$51,500","$73,305","$56,786",5538,15.98,885,1660,2226
9,15,11.00,Upper,No,122.20,"$51,500","$62,933","$48,750",6096,16.44,1002,1667,2398


Concatenación de tablas (iguales) descargadas desde internet...

In [48]:
urls = ["https://resultados.as.com/resultados/futbol/primera/2019_2020/",
        "https://resultados.as.com/resultados/futbol/primera/2020_2021/",
        "https://resultados.as.com/resultados/futbol/primera/2021_2022/"]

In [49]:
tablas = []
for a in range(len(urls)):
    print(a)
    html = requests.get(urls[a]).content
    df_list = pd.read_html(html)
    tablas.append(df_list[-1]) # Nos quedamos con la ultima tabla descargada


0
1
2


In [50]:
type (tablas)

list

In [51]:
tablas

[           Equipo  Pts.  PJ  PG  PE  PP
 0  1  Real Madrid    87  38  26   9   3
 1    2  Barcelona    82  38  25   7   6
 2     3  Atlético    70  38  18  16   4
 3      4  Sevilla    70  38  19  13   6
 4   5  Villarreal    60  38  18   6  14
 5  6  R. Sociedad    56  38  16   8  14
 6     18  Leganés    36  38   8  12  18
 7    19  Mallorca    33  38   9   6  23
 8    20  Espanyol    25  38   5  10  23,
                 Equipo  Pts.  PJ  PG  PE  PP
 0          1  Atlético    86  38  26   8   4
 1       2  Real Madrid    84  38  25   9   4
 2         3  Barcelona    79  38  24   7   7
 3           4  Sevilla    77  38  24   5   9
 4       5  R. Sociedad    62  38  17  11  10
 5             6  Betis    61  38  17  10  11
 6        18  Huesca  1    34  38   7  13  18
 7  19  Real Valladolid    31  38   5  16  17
 8            20  Eibar    30  38   6  12  20,
            Equipo  Pts.  PJ  PG  PE  PP
 0  1  Real Madrid    86  38  26   8   4
 1    2  Barcelona    73  38  21  10   7
 2   

In [52]:
tablas[0]

,Equipo,Pts.,PJ,PG,PE,PP
0,1 Real Madrid,87,38,26,9,3
1,2 Barcelona,82,38,25,7,6
2,3 Atlético,70,38,18,16,4
3,4 Sevilla,70,38,19,13,6
4,5 Villarreal,60,38,18,6,14
5,6 R. Sociedad,56,38,16,8,14
6,18 Leganés,36,38,8,12,18
7,19 Mallorca,33,38,9,6,23
8,20 Espanyol,25,38,5,10,23
